# Import python library

In [1]:
import os, shutil
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io, signal
from scipy.fftpack import fft, fftshift

import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler

print("Installed tensorflow version: ", tf.__version__)

Installed tensorflow version:  2.7.0


# GPU setting

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # GPU unit setting
tf.keras.backend.floatx()

'float32'

# Parameters Setting for training

In [3]:
### data parameters
winT = 18000 # one priod


### RMSprop optimizer parameters
lr_begin = 1e-3
RHO = 0.8

### Training parameters 
MAX_EPOCHS = 80
BATCH_SIZE = 128
EPOCH_1 = math.ceil(MAX_EPOCHS/2)
EPOCH_2 = math.ceil(MAX_EPOCHS*3/4)

# Load measured datasets

In [4]:
print("[Info.] where am I?")
%pwd

[Info.] where am I?


'/home/wkchoi/Arc-Fault/arc_fault'

In [5]:
# Define signal plot function
def show_signal_subplot(n_show, data, label, YRNG_MIN=-1, YRNG_MAX=1):
    plt.figure(2, figsize=(12, 8))
    plt.subplots_adjust(wspace=0.5)
    for i in range(n_show):
        plt.subplot(5, 10, i + 1)
        plt.plot(data[i], color='cornflowerblue')
        plt.text(int(data.shape[1]/2), 0, "%d" % label[i], fontsize=12, color='r')
        plt.xlim(0, data.shape[1])
        plt.ylim(YRNG_MIN, YRNG_MAX)
        
def data_shuffle(data):
    s = np.arange(data.shape[0])
    np.random.shuffle(s)
    
    return data[s]
    
    

def gen_one_period_data(file_path, winT):
    
    ##### load raw data
    data_raw = pd.read_excel(file_path, index_col=None, header=None, sheet_name='Sheet1')
    data = data_raw.to_numpy() # numpy array
    
    ##### Data Normalization
    data_n = np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]):
        data_n[:,i] = data[:,i]/max(abs(data[:,i]))  
    
    ##### expand one-period dataset
    dT = round(winT / 10)
    Num = round((data_n.shape[0] - winT)/dT)
    
    data_tot = np.zeros((winT,Num*data_n.shape[1]))
    index = 0
    for i in range(data_n.shape[1]):
        for j in range(Num):
            index = Num*i+j
            data_tot[:,index] = data_n[j*dT:winT+j*dT,i]
            
    print("[Info.] Total re-arranged one-period data shape: ", data_tot.shape)
    
    return data_tot

def merge_one_period_data(winT, *file_paths):
    
    subArrays = []
    for file_path in file_paths:
        data = gen_one_period_data(file_path, winT)
        subArrays.append(data)
                
    allArrays = np.concatenate(subArrays, axis=1)
    
    print("[Info.] ==========> Total merged one-period data shape: ", allArrays.shape, "\n")
        
    return allArrays

In [6]:
base_dir = "/home/wkchoi/Arc-Fault"

## 1. Fan dataset

In [7]:
### Fan arc raw dataset
file_path1 = os.path.join(base_dir, "Fan/arc/Fan_1_arc_humid_44(c61)_data.xlsx")
file_path2 = os.path.join(base_dir, "Fan/arc/Fan_2_arc_humid_45_data.xlsx")
file_path3 = os.path.join(base_dir, "Fan/arc/Fan_1_arc_humid_34(c14)_c2_data.xlsx")
file_path4 = os.path.join(base_dir, "Fan/arc/Fan_2_arc_humid_34(c23)_c15_data.xlsx")
file_path5 = os.path.join(base_dir, "Fan/arc/fan_1_arc_data.xlsx")
file_path6 = os.path.join(base_dir, "Fan/arc/fan_2_arc(c57)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4, file_path5, file_path6]
fan_arc = merge_one_period_data(winT, *file_paths)

##### Fan normal raw dataset
file_path1 = os.path.join(base_dir, "Fan/normal/Fan_1_normal_humid_room_data.xlsx")
file_path2 = os.path.join(base_dir, "Fan/normal/Fan_2_normal_humid_50_data.xlsx")
file_path3 = os.path.join(base_dir, "Fan/normal/fan_1_normal_data.xlsx")
file_path4 = os.path.join(base_dir, "Fan/normal/fan_2_normal(c54)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4]
fan_normal = merge_one_period_data(winT, *file_paths)

#### Fan Data shuffle
fan_arc_1 = data_shuffle(fan_arc.T)
fan_arc_2 = fan_arc_1.T[:,:]

fan_normal_1 = data_shuffle(fan_normal.T)
fan_normal_2 = fan_normal_1.T[:,:]

[Info.] Total re-arranged one-period data shape:  (18000, 1480)
[Info.] Total re-arranged one-period data shape:  (18000, 1560)
[Info.] Total re-arranged one-period data shape:  (18000, 80)
[Info.] Total re-arranged one-period data shape:  (18000, 600)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 1880)
[Info.] ==========> Total merged one-period data shape:  (18000, 7600) 

[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2160)
[Info.] ==========> Total merged one-period data shape:  (18000, 8160) 



## 2. Fluor dataset

In [8]:
### Fluor arc raw dataset
file_path1 = os.path.join(base_dir, "Fluor/arc/Fluor_1_arc_humid_40(c79)_data.xlsx")
file_path2 = os.path.join(base_dir, "Fluor/arc/Fluor_1_arc_humid_50(weird)_data.xlsx")
file_path3 = os.path.join(base_dir, "Fluor/arc/Fluor_2_arc_humid_40(c65)_data.xlsx")
file_path4 = os.path.join(base_dir, "Fluor/arc/Fluor_2_arc_humid_23(c51)_data(c45).xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4]
fluor_arc = merge_one_period_data(winT, *file_paths)

##### Fluor normal raw dataset
file_path1 = os.path.join(base_dir, "Fluor/normal/Fluor_1_normal_humid_room_data.xlsx")
file_path2 = os.path.join(base_dir, "Fluor/normal/Fluor_2_normal_humid_34_data.xlsx")
file_path3 = os.path.join(base_dir, "Fluor/normal/Fluor_2_normal(c37)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3]
fluor_normal = merge_one_period_data(winT, *file_paths)

#### Fluor Data shuffle
fluor_arc_1 = data_shuffle(fluor_arc.T)
fluor_arc_2 = fluor_arc_1.T[:,:]

fluor_normal_1 = data_shuffle(fluor_normal.T)
fluor_normal_2 = fluor_normal_1.T[:,:]

[Info.] Total re-arranged one-period data shape:  (18000, 3160)
[Info.] Total re-arranged one-period data shape:  (18000, 360)
[Info.] Total re-arranged one-period data shape:  (18000, 2600)
[Info.] Total re-arranged one-period data shape:  (18000, 1800)
[Info.] ==========> Total merged one-period data shape:  (18000, 7920) 

[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 1480)
[Info.] ==========> Total merged one-period data shape:  (18000, 5480) 



## 3. Heat dataset


In [9]:
### Heat arc raw dataset
file_path1 = os.path.join(base_dir, "Heat/arc/Heat_1_arc_humid_40(c56)_data.xlsx")
file_path2 = os.path.join(base_dir, "Heat/arc/Heat_1_arc_humid_50_data.xlsx")
file_path3 = os.path.join(base_dir, "Heat/arc/Heat_2_arc_humid_40_data.xlsx")
file_path4 = os.path.join(base_dir, "Heat/arc/Heater_2_arc_humid_34(c27)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4]
heat_arc = merge_one_period_data(winT, *file_paths)

##### Heat normal raw dataset
file_path1 = os.path.join(base_dir, "Heat/normal/Heat_1_normal_humid_room_data.xlsx")
file_path2 = os.path.join(base_dir, "Heat/normal/Heat_2_normal_humid_40_data.xlsx")
file_paths = [file_path1, file_path2]
heat_normal = merge_one_period_data(winT, *file_paths)

### Heat Data Shuffle
heat_arc_1 = data_shuffle(heat_arc.T)
heat_arc_2 = heat_arc_1.T[:,:]

heat_normal_1 = data_shuffle(heat_normal.T)
heat_normal_2 = heat_normal_1.T[:,:]

[Info.] Total re-arranged one-period data shape:  (18000, 2240)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 1080)
[Info.] ==========> Total merged one-period data shape:  (18000, 7320) 

[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] ==========> Total merged one-period data shape:  (18000, 4000) 



## 4. Incan dataset

In [10]:
### Incan arc raw dataset
file_path1 = os.path.join(base_dir, "incan/arc/incan_1_arc_humid_room_data.xlsx")
file_path2 = os.path.join(base_dir, "incan/arc/incan_2_arc_44_data.xlsx")
file_path3 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_40(c60)_data.xlsx")
file_path4 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_40(c61-c120)_data.xlsx")
file_path5 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_40(c121-c161)_data.xlsx")
file_path6 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_95(c60)_data.xlsx")
file_path7 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_95(c61-c82)_data.xlsx")
file_path8 = os.path.join(base_dir, "incan/arc/Incan_2_arc_humid_room(after humid95)(c53)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4, file_path5, file_path6,
             file_path7, file_path8]
incan_arc = merge_one_period_data(winT, *file_paths)

##### Incan normal raw dataset
file_path1 = os.path.join(base_dir, "incan/normal/incan_1_normal_humid_room_data.xlsx")
file_path2 = os.path.join(base_dir, "incan/normal/incan_2_normal_humid_room_data.xlsx")
file_paths = [file_path1, file_path2]
incan_normal = merge_one_period_data(winT, *file_paths)

### Incan Data Shuffle
incan_arc_1 = data_shuffle(incan_arc.T)
incan_arc_2 = incan_arc_1.T[:,:]

incan_normal_1 = data_shuffle(incan_normal.T)
incan_normal_2 = incan_normal_1.T[:,:]

[Info.] Total re-arranged one-period data shape:  (18000, 1880)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2400)
[Info.] Total re-arranged one-period data shape:  (18000, 2400)
[Info.] Total re-arranged one-period data shape:  (18000, 1640)
[Info.] Total re-arranged one-period data shape:  (18000, 2400)
[Info.] Total re-arranged one-period data shape:  (18000, 880)
[Info.] Total re-arranged one-period data shape:  (18000, 2120)
[Info.] ==========> Total merged one-period data shape:  (18000, 15720) 

[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] ==========> Total merged one-period data shape:  (18000, 4000) 



## 5. Led dataset

In [11]:
### LED arc raw dataset
file_path1 = os.path.join(base_dir, "Led/arc/led_1_arc_humid_23_c(30)_data(c29).xlsx")
file_path2 = os.path.join(base_dir, "Led/arc/LED_1_arc(c19)_data.xlsx")
file_path3 = os.path.join(base_dir, "Led/arc/LED_2_arc(c34)_data.xlsx")
file_path4 = os.path.join(base_dir, "Led/arc/led_1_arc(c43)_data.xlsx")
file_path5 = os.path.join(base_dir, "Led/arc/led_1_arc(c50)_data.xlsx")
file_path6 = os.path.join(base_dir, "Led/arc/led_2_arc(c56)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4, file_path5, file_path6]
led_arc = merge_one_period_data(winT, *file_paths)

##### LED normal raw dataset
file_path1 = os.path.join(base_dir, "Led/normal/led_1_normal_c(36)_data.xlsx")
file_path2 = os.path.join(base_dir, "Led/normal/led_1_normal_humid_room_data.xlsx")
file_path3 = os.path.join(base_dir, "Led/normal/led_2_normal_c(32)_data.xlsx")
file_path4 = os.path.join(base_dir, "Led/normal/LED_2_normal(c40)_data.xlsx")
file_path5 = os.path.join(base_dir, "Led/normal/led_1_normal(c3)_data.xlsx")
file_paths = [file_path1, file_path2, file_path3, file_path4, file_path5]
led_normal = merge_one_period_data(winT, *file_paths)

### LED Data Shuffle
led_arc_1 = data_shuffle(led_arc.T)
led_arc_2 = led_arc_1.T[:,:]

led_normal_1 = data_shuffle(led_normal.T)
led_normal_2 = led_normal_1.T[:,:]

[Info.] Total re-arranged one-period data shape:  (18000, 1160)
[Info.] Total re-arranged one-period data shape:  (18000, 760)
[Info.] Total re-arranged one-period data shape:  (18000, 1360)
[Info.] Total re-arranged one-period data shape:  (18000, 1600)
[Info.] Total re-arranged one-period data shape:  (18000, 1800)
[Info.] Total re-arranged one-period data shape:  (18000, 1920)
[Info.] ==========> Total merged one-period data shape:  (18000, 8600) 

[Info.] Total re-arranged one-period data shape:  (18000, 1440)
[Info.] Total re-arranged one-period data shape:  (18000, 2000)
[Info.] Total re-arranged one-period data shape:  (18000, 1280)
[Info.] Total re-arranged one-period data shape:  (18000, 1600)
[Info.] Total re-arranged one-period data shape:  (18000, 120)
[Info.] ==========> Total merged one-period data shape:  (18000, 6440) 



In [12]:
# 데이터 불균형으로 데이터를 자름 (oversampling 또는 다른 방법 필요)
print('------Fan------')
print(fan_arc_2.shape)
print(fan_normal_2.shape)
fan_normal_2 = fan_normal_1.T[:,:7600]
print(fan_arc_2.shape)
print(fan_normal_2.shape)

print('------Fluor------')
print(fluor_arc_2.shape)
print(fluor_normal_2.shape)
fluor_arc_2 = fluor_arc_1.T[:,:5480]
print(fluor_arc_2.shape)
print(fluor_normal_2.shape)

print('------Heat------')
print(heat_arc_2.shape)
print(heat_normal_2.shape)
heat_arc_2 = heat_arc_1.T[:,:4000]
print(heat_arc_2.shape)
print(heat_normal_2.shape)

print('------Incan------')
print(incan_arc_2.shape)
print(incan_normal_2.shape)
incan_arc_2 = incan_arc_1.T[:,:4000]
print(incan_arc_2.shape)
print(incan_normal_2.shape)

print('------Led------')
print(led_arc_2.shape)
print(led_normal_2.shape)
led_arc_2 = led_arc_1.T[:,:6440]
print(led_arc_2.shape)
print(led_normal_2.shape)

------Fan------
(18000, 7600)
(18000, 8160)
(18000, 7600)
(18000, 7600)
------Fluor------
(18000, 7920)
(18000, 5480)
(18000, 5480)
(18000, 5480)
------Heat------
(18000, 7320)
(18000, 4000)
(18000, 4000)
(18000, 4000)
------Incan------
(18000, 15720)
(18000, 4000)
(18000, 4000)
(18000, 4000)
------Led------
(18000, 8600)
(18000, 6440)
(18000, 6440)
(18000, 6440)


# Feature Extraction

In [14]:
# 특성추출을 위한 함수 작성 ( 변수이름, 데이터 크기)
def create_feature(x, y):
    feature = np.zeros(shape=(39,y))
    mean_val = np.zeros((10, y))
    std_val = np.zeros((10, y))
    Pole_diff = np.zeros((10, y))
    Avg_diff = np.zeros((9, y))

    for i in range(y):
        sig = x[:, i]
        win = round(x.shape[0]/10)
        for j in range(10):
            tmp = x[int(j * win) : int((j + 1) * win)]
            mean_val[j] = np.mean(tmp[j],axis=0)
            std_val[j] = np.std(tmp[j])
            Pole_diff[j] = np.max(tmp[j]) - np.min(tmp[j])
        for z in range(9):
            Avg_diff[z] = np.mean(tmp[z], axis=0) - np.mean(tmp[z+1],axis=0)
        
    feature = np.concatenate((mean_val, std_val, Pole_diff, Avg_diff), axis =0)
    return feature

In [15]:
# 5종류 Arc / Normal 특성추출
fan_arc = create_feature(fan_arc_2, 7600)
fan_normal = create_feature(fan_normal_2, 7600)
fluor_arc = create_feature(fluor_arc_2, 5480)
fluor_normal = create_feature(fluor_normal_2, 5480)
heat_arc = create_feature(heat_arc_2, 4000)
heat_normal = create_feature(heat_normal_2, 4000)
incan_arc = create_feature(incan_arc_2, 4000)
incan_normal = create_feature(incan_normal_2, 4000)
led_arc = create_feature(led_arc_2, 6440)
led_normal = create_feature(led_normal_2, 6440)


In [16]:
# 특성추출한 데이터 크기 확인
print('------Fan------')
print(fan_arc.shape)
print(fan_normal.shape)

print('------Fluor------')
print(fluor_arc.shape)
print(fluor_normal.shape)

print('------Heat------')
print(heat_arc.shape)
print(heat_normal.shape)


print('------Incan------')
print(incan_arc.shape)
print(incan_normal.shape)

print('------Led------')
print(led_arc.shape)
print(led_normal.shape)

------Fan------
(39, 7600)
(39, 7600)
------Fluor------
(39, 5480)
(39, 5480)
------Heat------
(39, 4000)
(39, 4000)
------Incan------
(39, 4000)
(39, 4000)
------Led------
(39, 6440)
(39, 6440)


# Data Labeling

In [17]:
##### Fan Labeling
norm_fan_label = np.zeros(fan_normal.shape[1]) # label 0
arc_fan_label = np.ones(fan_arc.shape[1])  # label 1

##### Fluor Labeling
norm_fluor_label = 2*np.ones(fluor_normal.shape[1]) # label 2
arc_fluor_label = 3*np.ones(fluor_arc.shape[1])  # label 3

##### Heat Labeling
norm_heat_label = 4*np.ones(heat_normal.shape[1]) # label 4
arc_heat_label = 5*np.ones(heat_arc.shape[1])  # label 5

##### Incan Labeling
norm_incan_label = 6*np.ones(incan_normal.shape[1]) # label 6
arc_incan_label = 7*np.ones(incan_arc.shape[1])  # label 7

##### LED Labeling
norm_led_label = 8*np.ones(led_normal.shape[1]) # label 8
arc_led_label = 9*np.ones(led_arc.shape[1])  # label 9

# Data Shuffling

In [18]:
##### Prepare full datasets for training
Train_raw = np.concatenate((fan_normal.T, fan_arc.T, fluor_normal.T, fluor_arc.T,
                           heat_normal.T, heat_arc.T, incan_normal.T, incan_arc.T, 
                           led_normal.T, led_arc.T), axis=0)
Train_label = np.concatenate((norm_fan_label, arc_fan_label, norm_fluor_label, arc_fluor_label,
                             norm_heat_label, arc_heat_label, norm_incan_label, arc_incan_label,
                             norm_led_label, arc_led_label))
print('[Info.] Total dataset shape: ', Train_raw.shape)

[Info.] Total dataset shape:  (55040, 39)


In [19]:
# s라는 배열에 data의 인덱스를 넣고 섞은 뒤 data와 data_label 배열에 대입.(train)
s = np.arange(Train_raw.shape[0])
np.random.shuffle(s)

data_tr = Train_raw[s]
data_tr_label = Train_label[s]
print("===> Display label numbers after data shuffling:", "\n", data_tr_label[0:100])

===> Display label numbers after data shuffling: 
 [0. 0. 5. 0. 3. 1. 1. 7. 3. 7. 0. 6. 0. 1. 2. 3. 7. 4. 2. 7. 7. 2. 1. 6.
 0. 8. 3. 5. 5. 0. 1. 2. 6. 9. 1. 1. 3. 3. 0. 3. 3. 6. 7. 3. 8. 0. 2. 1.
 3. 6. 3. 0. 1. 7. 7. 2. 2. 3. 8. 9. 8. 2. 6. 9. 9. 8. 9. 0. 1. 2. 7. 6.
 9. 0. 9. 6. 2. 8. 3. 4. 1. 7. 4. 6. 0. 8. 9. 4. 7. 5. 7. 2. 8. 2. 0. 1.
 9. 8. 8. 4.]


# Construct Train and Testing dataset


In [20]:
SPL_RATIO = 0.8 
index = round(data_tr.shape[0]*SPL_RATIO)

### train 데이터와 test 데이터로 분리
train_X, train_Y = data_tr[:index], data_tr_label[:index]
test_X, test_Y = data_tr[index:], data_tr_label[index:]
print('[Info.] Number of training dataset: ', len(train_X))
print('[Info.] Number of test dataset: ', len(test_X))

### Save trainind and test datasets and labels
dataset_dir = './TrTedata'
if os.path.exists(dataset_dir):  # 반복적인 실행을 위해 디렉토리를 삭제
    shutil.rmtree(dataset_dir)   
os.mkdir(dataset_dir)

# training
train_X_dir = os.path.join(dataset_dir, 'train_X.npy')
np.save(train_X_dir, train_X)

train_Y_dir = os.path.join(dataset_dir, 'train_Y.npy')
np.save(train_Y_dir, train_Y)

# test
test_X_dir = os.path.join(dataset_dir, 'test_X.npy')
np.save(test_X_dir, test_X)

test_Y_dir = os.path.join(dataset_dir, 'test_Y.npy')
np.save(test_Y_dir, test_Y)

print(train_X.shape)
print(test_X.shape)
print(train_Y.shape)
print(test_Y.shape)

[Info.] Number of training dataset:  44032
[Info.] Number of test dataset:  11008
(44032, 39)
(11008, 39)
(44032,)
(11008,)


In [21]:
### Training and test datasets
x_train = np.reshape(train_X, (len(train_X),train_X.shape[1],1))
x_test = np.reshape(test_X, (len(test_X),train_X.shape[1],1))

### Training and test labeling
y_train = np.asarray(train_Y).astype('float32')
y_test = np.asarray(test_Y).astype('float32')

print('[Info.] x_train numpy shape: ', x_train.shape)
print('[Info.] x_test numpy shape: ', x_test.shape)
print('[Info.] y_train numpy shape: ', y_train.shape)
print('[Info.] y_test numpy shape: ', y_test.shape)

[Info.] x_train numpy shape:  (44032, 39, 1)
[Info.] x_test numpy shape:  (11008, 39, 1)
[Info.] y_train numpy shape:  (44032,)
[Info.] y_test numpy shape:  (11008,)


In [23]:
x_train = x_train.reshape((44032, 39))
x_test = x_test.reshape((11008, 39))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(44032, 39)
(11008, 39)
(44032,)
(11008,)


#  Model Compile

### Random Forest

- 앙상블 학습의 대표 중 하나로 안정적인 성능 덕분에 널리 사용.
- 각 트리를 훈련하기 위한 데이터를 랜덤하게 만든다.
- 이 데이터를 만드는 방법은 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다. 이때 한 샘플이 중복되어 추출.(부트스트랩 샘플)
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음.
- RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택.
- 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻음.
-

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

# Model Training

In [25]:
rf.fit(x_train, y_train)

RandomForestClassifier()

# Evaluate

In [26]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % rf.score(x_train, y_train))
print('시험용 데이터로 측정한 정확도 = %.2f' % rf.score(x_test, y_test))

학습용 데이터로 측정한 정확도 = 1.00
시험용 데이터로 측정한 정확도 = 1.00


In [27]:
y_pred = rf.predict(x_test)

print("테스트 세트에 대한 예측값:\n {}".format(y_pred))
print("테스트 세트의 정확도: {:.2f}".format(np.mean(y_pred == y_test)))

테스트 세트에 대한 예측값:
 [1. 8. 2. ... 3. 3. 0.]
테스트 세트의 정확도: 1.00


In [28]:
from sklearn.model_selection import GridSearchCV, KFold

params = {'n_estimators':[10, 100], 
         'max_depth': [6, 8, 10, 12],
         'min_samples_leaf' : [8, 12, 18],
         'min_samples_split': [8, 16, 20]
         }
rf = RandomForestClassifier(random_state=0, n_jobs= -1)
gs = GridSearchCV(rf, params, cv= KFold(n_splits=2, shuffle=True), scoring='accuracy', verbose = 3)
gs.fit(x_train, y_train)

print("최적 하이퍼 파라미터: ", gs.best_params_)
print("최고 예측 정확도: {:.4f}".format(gs.best_score_))

Fitting 2 folds for each of 72 candidates, totalling 144 fits
[CV 1/2] END max_depth=6, min_samples_leaf=8, min_samples_split=8, n_estimators=10;, score=0.853 total time=   0.7s
[CV 2/2] END max_depth=6, min_samples_leaf=8, min_samples_split=8, n_estimators=10;, score=0.781 total time=   0.4s
[CV 1/2] END max_depth=6, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=1.000 total time=   0.5s
[CV 2/2] END max_depth=6, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=1.000 total time=   0.4s
[CV 1/2] END max_depth=6, min_samples_leaf=8, min_samples_split=16, n_estimators=10;, score=0.853 total time=   0.2s
[CV 2/2] END max_depth=6, min_samples_leaf=8, min_samples_split=16, n_estimators=10;, score=0.781 total time=   0.2s
[CV 1/2] END max_depth=6, min_samples_leaf=8, min_samples_split=16, n_estimators=100;, score=1.000 total time=   0.4s
[CV 2/2] END max_depth=6, min_samples_leaf=8, min_samples_split=16, n_estimators=100;, score=1.000 total time=   0.3s
[C

[CV 1/2] END max_depth=8, min_samples_leaf=18, min_samples_split=20, n_estimators=100;, score=1.000 total time=   0.3s
[CV 2/2] END max_depth=8, min_samples_leaf=18, min_samples_split=20, n_estimators=100;, score=1.000 total time=   0.3s
[CV 1/2] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=10;, score=1.000 total time=   0.2s
[CV 2/2] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=10;, score=1.000 total time=   0.2s
[CV 1/2] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=1.000 total time=   0.3s
[CV 2/2] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=1.000 total time=   0.3s
[CV 1/2] END max_depth=10, min_samples_leaf=8, min_samples_split=16, n_estimators=10;, score=1.000 total time=   0.2s
[CV 2/2] END max_depth=10, min_samples_leaf=8, min_samples_split=16, n_estimators=10;, score=1.000 total time=   0.2s
[CV 1/2] END max_depth=10, min_samples_leaf=8, min_sampl

[CV 2/2] END max_depth=12, min_samples_leaf=18, min_samples_split=16, n_estimators=100;, score=1.000 total time=   0.3s
[CV 1/2] END max_depth=12, min_samples_leaf=18, min_samples_split=20, n_estimators=10;, score=1.000 total time=   0.2s
[CV 2/2] END max_depth=12, min_samples_leaf=18, min_samples_split=20, n_estimators=10;, score=1.000 total time=   0.2s
[CV 1/2] END max_depth=12, min_samples_leaf=18, min_samples_split=20, n_estimators=100;, score=1.000 total time=   0.3s
[CV 2/2] END max_depth=12, min_samples_leaf=18, min_samples_split=20, n_estimators=100;, score=1.000 total time=   0.4s
최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 1.0000


In [29]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 6, min_samples_leaf = 8, min_samples_split = 8, n_estimators=100)

In [30]:
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, min_samples_split=8)

In [31]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % rf.score(x_train, y_train))
print('시험용 데이터로 측정한 정확도 = %.2f' % rf.score(x_test, y_test))

학습용 데이터로 측정한 정확도 = 1.00
시험용 데이터로 측정한 정확도 = 1.00


### KNN

In [32]:
# n_neighbors : 분류 시 고려할 인접 샘플 수
# weights(default='uniform') : 'distance'로 설정하면, 분류할 때 인접한 샘플의 거리에 따라 다른 가중치 부여 (가까울수록 큰 가중치)
# metric(default='minkowski') : 거리계산의 척도

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, weights="distance", metric="euclidean")

In [33]:
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=1, weights='distance')

In [34]:
y_pred = knn.predict(x_test)

print("테스트 세트에 대한 예측값:\n {}".format(y_pred))
print("테스트 세트의 정확도: {:.2f}".format(np.mean(y_pred == y_test)))

테스트 세트에 대한 예측값:
 [1. 8. 2. ... 3. 3. 0.]
테스트 세트의 정확도: 1.00


In [35]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, weights="distance", metric="euclidean")
param_grid = [{'n_neighbors': range(1,10)}]
gs = GridSearchCV(knn, param_grid, cv= KFold(n_splits=5, shuffle=True), scoring='accuracy', verbose = 3)
gs.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .....................n_neighbors=1;, score=1.000 total time=   1.9s
[CV 2/5] END .....................n_neighbors=1;, score=1.000 total time=   1.8s
[CV 3/5] END .....................n_neighbors=1;, score=1.000 total time=   1.8s
[CV 4/5] END .....................n_neighbors=1;, score=1.000 total time=   1.8s
[CV 5/5] END .....................n_neighbors=1;, score=1.000 total time=   1.8s
[CV 1/5] END .....................n_neighbors=2;, score=1.000 total time=   2.1s
[CV 2/5] END .....................n_neighbors=2;, score=1.000 total time=   2.1s
[CV 3/5] END .....................n_neighbors=2;, score=1.000 total time=   2.1s
[CV 4/5] END .....................n_neighbors=2;, score=1.000 total time=   2.1s
[CV 5/5] END .....................n_neighbors=2;, score=1.000 total time=   2.1s
[CV 1/5] END .....................n_neighbors=3;, score=1.000 total time=   2.3s
[CV 2/5] END .....................n_neighbors=3;,

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=KNeighborsClassifier(metric='euclidean', n_neighbors=1,
                                            weights='distance'),
             param_grid=[{'n_neighbors': range(1, 10)}], scoring='accuracy',
             verbose=3)

In [37]:
print("최적 하이퍼 파라미터: ", gs.best_params_)
print("최고 예측 정확도: {:.4f}".format(gs.best_score_))

최적 하이퍼 파라미터:  {'n_neighbors': 1}
최고 예측 정확도: 1.0000


In [38]:
knn = KNeighborsClassifier(n_neighbors=1, weights="distance", metric="euclidean")

In [39]:
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=1, weights='distance')

In [40]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % knn.score(x_train, y_train))
print('시험용 데이터로 측정한 정확도 = %.2f' % knn.score(x_test, y_test))

학습용 데이터로 측정한 정확도 = 1.00
시험용 데이터로 측정한 정확도 = 1.00


### Extra Trees

- 랜덤 포레스트와 비슷하게 동작.
- 결정 트리가 제공하는 대부분의 매개변수를 지원. 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용.

In [41]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, x_train, y_train, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 1.0


In [42]:
et.fit(x_train, y_train)
print(et.feature_importances_)

[0.01699946 0.00962231 0.02062749 0.02993479 0.03587629 0.04797844
 0.03952264 0.03825744 0.02660077 0.02727088 0.02230311 0.01615028
 0.01881748 0.01879772 0.01379352 0.01851568 0.01219435 0.01170963
 0.02388932 0.01678792 0.02963995 0.01814423 0.02873368 0.03587623
 0.03610251 0.02678946 0.02382473 0.0304536  0.0277167  0.02655013
 0.0497368  0.02237522 0.00783151 0.0395558  0.01704862 0.05043943
 0.01046386 0.02611774 0.02695028]


In [43]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % et.score(x_train, y_train))
print('시험용 데이터로 측정한 정확도 = %.2f' % et.score(x_test, y_test))

학습용 데이터로 측정한 정확도 = 1.00
시험용 데이터로 측정한 정확도 = 1.00
